<a href="https://colab.research.google.com/github/mveerara/Teleradiology/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import imageio
from os import listdir
import skimage.transform
import pickle
import sys, os
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/MyDrive')

Mounted at /MyDrive


In [ ]:
!pwd

/content


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
zip_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/images/images.zip'
!cp "{zip_path}" .
!unzip -q images.zip
!rm images.zip

^C
[images.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of images.zip or
        images.zip.zip, and cannot find images.zip.ZIP, period.


In [ ]:
image_folder_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/images/images' #'/content/images' #sys.argv[1] # folder contain all image


In [ ]:
data_entry_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/Data_Entry_2017_v2020.csv' #sys.argv[2] 
bbox_list_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/BBox_List_2017.csv' #sys.argv[3]
train_txt_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/train_val_list.txt' #sys.argv[4]
valid_txt_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/test_list.txt' #sys.argv[5]
data_path = '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/output' #sys.argv[6] # ouput folder for preprocessed data

In [ ]:
image_folder_path,data_entry_path,bbox_list_path,train_txt_path,valid_txt_path,data_path

('/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/images/images',
 '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/Data_Entry_2017_v2020.csv',
 '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/BBox_List_2017.csv',
 '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/train_val_list.txt',
 '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/test_list.txt',
 '/MyDrive/MyDrive/Teleradiology/Example_Program/example1/Dataset/output')

In [ ]:
def get_labels(pic_id):
    labels = meta_data.loc[meta_data["Image Index"]==pic_id,"Finding Labels"]
    return labels.tolist()[0].split("|")

In [ ]:
# load data
meta_data = pd.read_csv(data_entry_path)
bbox_list = pd.read_csv(bbox_list_path)
with open(train_txt_path, "r") as f:
    train_list = [ i.strip() for i in f.readlines()]
with open(valid_txt_path, "r") as f:
    valid_list = [ i.strip() for i in f.readlines()]
label_eight = list(np.unique(bbox_list["Finding Label"])) + ["No Finding"]

In [ ]:
!ls 

In [ ]:
# transform training images
print("training example:",len(train_list))
print("take care of your RAM here !!!")
train_X = []
for i in range(len(train_list[0:2000])): # only selecting 2000 images 
    image_path = os.path.join(image_folder_path,train_list[i])
    try:
        img = imageio.imread(image_path)#for filefolder in os.listdir(data_folder):      
        #filefolder = data_folder + "\\" + filefolder
    except OSError:
        pass
    #img = imageio.imread(image_path)
    if img.shape != (1024,1024): # there some image with shape (1024,1024,4) in training set
        img = img[:,:,0]
    img_resized = skimage.transform.resize(img,(256,256)) # or use img[::4] here
    train_X.append((np.array(img_resized)/255).reshape(256,256,1))
    if i % 3000==0:
        print(i)

training example: 86524
take care of your RAM here !!!
0


KeyboardInterrupt: ignored

In [ ]:
print(len(train_X))

2000


In [ ]:
train_X = np.array(train_X)
np.save(os.path.join(data_path,"train_X_small.npy"), train_X)

In [ ]:
# transform validation images
print("validation example:",len(valid_list))
valid_X = []
for i in range(len(valid_list[0:2000])):
    image_path = os.path.join(image_folder_path,valid_list[i])
    try:
        img = imageio.imread(image_path)#for filefolder in os.listdir(data_folder):      
        #filefolder = data_folder + "\\" + filefolder
    except OSError:
        pass
    #img = imageio.imread(image_path)
    if img.shape != (1024,1024):
        img = img[:,:,0]
    img_resized = skimage.transform.resize(img,(256,256))
#     if img.shape != (1024,1024):
#             train_X.append(img[:,:,0])
#     else:
    valid_X.append((np.array(img_resized)/255).reshape(256,256,1))
    if i % 3000==0:
        print(i)

validation example: 25596
0


In [ ]:
valid_X = np.array(valid_X)
np.save(os.path.join(data_path,"valid_X_small.npy"), valid_X)

In [ ]:
# process label
print("label preprocessing")

train_y = []
for train_id in train_list[0:2000]:
    train_y.append(get_labels(train_id))
valid_y = []
for valid_id in valid_list[0:2000]:
    valid_y.append(get_labels(valid_id))

label preprocessing


In [ ]:
encoder = MultiLabelBinarizer()
encoder.fit(train_y+valid_y)
train_y_onehot = encoder.transform(train_y)
valid_y_onehot = encoder.transform(valid_y)
train_y_onehot = np.delete(train_y_onehot, [2,3,5,6,7,10,12],1) # delete out 8 and "No Finding" column
valid_y_onehot = np.delete(valid_y_onehot, [2,3,5,6,7,10,12],1) # delete out 8 and "No Finding" column

In [ ]:
with open(data_path + "/train_y_onehot.pkl","wb") as f:
    pickle.dump(train_y_onehot, f)
with open(data_path + "/valid_y_onehot.pkl","wb") as f:
    pickle.dump(valid_y_onehot, f)
with open(data_path + "/label_encoder.pkl","wb") as f:
    pickle.dump(encoder, f)